In [1]:
import sys, os
if 'google.colab' in sys.modules and not os.path.exists('.setup_complete'):
    !wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/setup_colab.sh -O- | bash
    !wget -q https://raw.githubusercontent.com/yandexdataschool/Practical_RL/master/week02_value_based/mdp.py
    !touch .setup_complete

# This code creates a virtual display to draw game images on.
# It will have no effect if your machine has a monitor.
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY")) == 0:
    !bash ../xvfb start
    os.environ['DISPLAY'] = ':1'

Starting virtual X frame buffer: Xvfb.


# HW Part 1: Value iteration convergence

### Find an MDP for which value iteration takes long to converge  (1 pts)

When we ran value iteration on the small frozen lake problem, the last iteration where an action changed was iteration 6--i.e., value iteration computed the optimal policy at iteration 6. Are there any guarantees regarding how many iterations it'll take value iteration to compute the optimal policy? There are no such guarantees without additional assumptions--we can construct the MDP in such a way that the greedy policy will change after arbitrarily many iterations.

Your task: define an MDP with at most 3 states and 2 actions, such that when you run value iteration, the optimal action changes at iteration >= 50. Use discount=0.95. (However, note that the discount doesn't matter here--you can construct an appropriate MDP with any discount.)

Note: value function must change at least once after iteration >=50, not necessarily change on every iteration till >=50.

In [2]:
transition_probs = {
    's0': {
        'a0': {'s0': 1},
    },
    's1': {
        'a0': {'s2': 1},
        'a1': {'s0': 1}
    },
    's2': {
        'a0': {'s2': 1},
    }
}
rewards = {
    's1': {'a1': {'s0': 9.47}},
    's2': {'a0': {'s2': 1}},
}

from mdp import MDP
from mdp import FrozenLakeEnv
from numpy import random
import numpy as np
mdp = MDP(transition_probs, rewards, initial_state='s1')
# Feel free to change the initial_state

In [3]:
def get_action_value(mdp, state_values, state, action, gamma):
    """ Computes Q(s,a) as in formula above """

    q = 0
    for next_state, proba in mdp.get_next_states(state, action).items():
        q += proba * (mdp.get_reward(state, action, next_state) + gamma * state_values[next_state])

    return q

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
def get_new_state_value(mdp, state_values, state, gamma):
    """ Computes next V(s) as in formula above. Please do not change state_values in process. """
    if mdp.is_terminal(state):
        return 0

    return max(get_action_value(mdp, state_values, state, action, gamma) for action in mdp.get_possible_actions(state))

In [5]:
def get_optimal_action(mdp, state_values, state, gamma=0.9):
    """ Finds optimal action using formula above. """
    if mdp.is_terminal(state):
        return None

    actions = mdp.get_possible_actions(state)
    indx = np.argmax([get_action_value(mdp, state_values, state, action, gamma) for action in actions])
    return actions[indx]

In [6]:
def value_iteration(mdp, state_values=None, gamma=0.9, num_iter=1000, min_difference=1e-5):
    """ performs num_iter value iteration steps starting from state_values. Same as before but in a function """
    state_values = state_values or {s: 0 for s in mdp.get_all_states()}
    for i in range(num_iter):

        # Compute new state values using the functions you defined above. It must be a dict {state : new_V(state)}
        new_state_values = {state : get_new_state_value(mdp, state_values, state, gamma) for state in mdp.get_all_states()}

        assert isinstance(new_state_values, dict)

        # Compute difference
        diff = max(abs(new_state_values[s] - state_values[s])
                   for s in mdp.get_all_states())

        print("iter %4i   |   diff: %6.5f   |   " % (i, diff), end="")
        print('   '.join("V(%s) = %.3f" % (s, v) for s, v in state_values.items()))

        state_values = new_state_values
        if diff < min_difference:
            break

    return state_values

In [7]:
gamma = 0.95
state_values = {s: 0 for s in mdp.get_all_states()}
policy = np.array([get_optimal_action(mdp, state_values, state, gamma)
                   for state in sorted(mdp.get_all_states())])

for i in range(100):
    print("after iteration %i" % i)
    state_values = value_iteration(mdp, state_values, num_iter=1)

    new_policy = np.array([get_optimal_action(mdp, state_values, state, gamma)
                           for state in sorted(mdp.get_all_states())])

    n_changes = (policy != new_policy).sum()
    print(policy)
    print("N actions changed = %i \n" % n_changes)
    policy = new_policy

# please ignore iter 0 at each step

after iteration 0
iter    0   |   diff: 9.47000   |   V(s0) = 0.000   V(s1) = 0.000   V(s2) = 0.000
['a0' 'a1' 'a0']
N actions changed = 0 

after iteration 1
iter    0   |   diff: 0.90000   |   V(s0) = 0.000   V(s1) = 9.470   V(s2) = 1.000
['a0' 'a1' 'a0']
N actions changed = 0 

after iteration 2
iter    0   |   diff: 0.81000   |   V(s0) = 0.000   V(s1) = 9.470   V(s2) = 1.900
['a0' 'a1' 'a0']
N actions changed = 0 

after iteration 3
iter    0   |   diff: 0.72900   |   V(s0) = 0.000   V(s1) = 9.470   V(s2) = 2.710
['a0' 'a1' 'a0']
N actions changed = 0 

after iteration 4
iter    0   |   diff: 0.65610   |   V(s0) = 0.000   V(s1) = 9.470   V(s2) = 3.439
['a0' 'a1' 'a0']
N actions changed = 0 

after iteration 5
iter    0   |   diff: 0.59049   |   V(s0) = 0.000   V(s1) = 9.470   V(s2) = 4.095
['a0' 'a1' 'a0']
N actions changed = 0 

after iteration 6
iter    0   |   diff: 0.53144   |   V(s0) = 0.000   V(s1) = 9.470   V(s2) = 4.686
['a0' 'a1' 'a0']
N actions changed = 0 

after iterati

### Value iteration convervence proof (1 pts)
**Note:** Assume that $\mathcal{S}, \mathcal{A}$ are finite.

Update of value function in value iteration can be rewritten in a form of Bellman operator:

$$(TV)(s) = \max_{a \in \mathcal{A}}\mathbb{E}\left[ r_{t+1} + \gamma V(s_{t+1}) | s_t = s, a_t = a\right]$$

Value iteration algorithm with Bellman operator:

---
&nbsp;&nbsp; Initialize $V_0$

&nbsp;&nbsp; **for** $k = 0,1,2,...$ **do**

&nbsp;&nbsp;&nbsp;&nbsp; $V_{k+1} \leftarrow TV_k$

&nbsp;&nbsp;**end for**

---

In [lecture](https://docs.google.com/presentation/d/1lz2oIUTvd2MHWKEQSH8hquS66oe4MZ_eRvVViZs2uuE/edit#slide=id.g4fd6bae29e_2_4) we established contraction property of bellman operator:

$$
||TV - TU||_{\infty} \le \gamma ||V - U||_{\infty}
$$

For all $V, U$

Using contraction property of Bellman operator, Banach fixed-point theorem and Bellman equations prove that value function converges to $V^*$ in value iterateion

Proof.
$$
\left\|V_k-V^*\right\|_{\infty}=\left\|T V_{k-1}-TV^*\right\|_{\infty} \leq \gamma\left\|V_{k-1}-V^*\right\|_{\infty} \leq \cdots \leq \gamma^k\left\|V_0-V^*\right\|_{\infty}
$$
При $k \rightarrow \infty$
$$\left\|V_k-V^*\right\|_{\infty} \rightarrow 0$$
Отсюда $$\lim _{k \rightarrow \infty} V_k=V^*$$


### Asynchronious value iteration (2 pts)

Consider the following algorithm:

---

Initialize $V_0$

**for** $k = 0,1,2,...$ **do**

&nbsp;&nbsp;&nbsp;&nbsp; Select some state $s_k \in \mathcal{S}$    

&nbsp;&nbsp;&nbsp;&nbsp; $V(s_k) := (TV)(s_k)$

**end for**

---


Note that unlike common value iteration, here we update only a single state at a time.

**Homework.** Prove the following proposition:

If for all $s \in \mathcal{S}$, $s$ appears in the sequence $(s_0, s_1, ...)$ infinitely often, then $V$ converges to $V*$

Proof.

$\left\|V_{t+1}-V^*\right\|_{\infty} \leq \left\|V_{t+1}-V^*\right\|_1 = \sum_{i=1}^{|\mathcal{S}|} \left|V_{t+1}\left(s_i\right) - V^*\left(s_i\right)\right|$
Выделим подпоследовательность $s_{k_n}$ состоящую из одинаковых состояний, $n\rightarrow\infty$ Для нее задача сводится к обычному value iteration c одним состоянием и $$\left|V_n(s)-V^*(s)\right| \rightarrow 0$$ $$\lim _{n \rightarrow \infty} V_n(s)=V^*(s)$$

Тогда при $k \rightarrow \infty$
$$\left\|V_{t+1}-V^*\right\|_{\infty} \leq \sum_{i=1}^{|\mathcal{S}|} \left|V_{t+1}\left(s_i\right) - V^*\left(s_i\right)\right| \rightarrow 0$$

$$\lim _{k \rightarrow \infty} V_k=V^*$$


# HW Part 2: Policy iteration

## Policy iteration implementateion (3 pts)

Let's implement exact policy iteration (PI), which has the following pseudocode:

---
Initialize $\pi_0$   `// random or fixed action`

For $n=0, 1, 2, \dots$
- Compute the state-value function $V^{\pi_{n}}$
- Using $V^{\pi_{n}}$, compute the state-action-value function $Q^{\pi_{n}}$
- Compute new policy $\pi_{n+1}(s) = \operatorname*{argmax}_a Q^{\pi_{n}}(s,a)$
---

Unlike VI, policy iteration has to maintain a policy - chosen actions from all states - and estimate $V^{\pi_{n}}$ based on this policy. It only changes policy once values converged.


Below are a few helpers that you may or may not use in your implementation.

In [8]:
transition_probs = {
    's0': {
        'a0': {'s0': 0.5, 's2': 0.5},
        'a1': {'s2': 1}
    },
    's1': {
        'a0': {'s0': 0.7, 's1': 0.1, 's2': 0.2},
        'a1': {'s1': 0.95, 's2': 0.05}
    },
    's2': {
        'a0': {'s0': 0.4, 's1': 0.6},
        'a1': {'s0': 0.3, 's1': 0.3, 's2': 0.4}
    }
}
rewards = {
    's1': {'a0': {'s0': +5}},
    's2': {'a1': {'s0': -1}}
}

from mdp import MDP
mdp = MDP(transition_probs, rewards, initial_state='s0')

Let's write a function called `compute_vpi` that computes the state-value function $V^{\pi}$ for an arbitrary policy $\pi$.

Unlike VI, this time you must find the exact solution, not just a single iteration.

Recall that $V^{\pi}$ satisfies the following linear equation:
$$V^{\pi}(s) = \sum_{s'} P(s,\pi(s),s')[ R(s,\pi(s),s') + \gamma V^{\pi}(s')]$$

You'll have to solve a linear system in your code. (Find an exact solution, e.g., with `np.linalg.solve`.)

In [9]:
def compute_vpi(mdp, policy, gamma):
    """
    Computes V^pi(s) FOR ALL STATES under given policy.
    :param policy: a dict of currently chosen actions {s : a}
    :returns: a dict {state : V^pi(state) for all states}
    """
    state_to_idx = {s: i for i, s in enumerate(mdp.get_all_states())}
    n_states = len(mdp.get_all_states())

    A = np.zeros((n_states, n_states))
    B = np.zeros(n_states)

    for state in mdp.get_all_states():
        if policy[state] is not None:
            for next_state, proba in mdp.get_next_states(state, policy[state]).items():
                A[state_to_idx[state], state_to_idx[next_state]] -= proba * gamma
                B[state_to_idx[state]] += proba * mdp.get_reward(state, policy[state], next_state)

        A[state_to_idx[state], state_to_idx[state]] += 1

    value_states = np.linalg.solve(A, B)
    return {s : value_states[state_to_idx[s]] for s in mdp.get_all_states()}

In [10]:
test_policy = {s: np.random.choice(
    mdp.get_possible_actions(s)) for s in mdp.get_all_states()}
new_vpi = compute_vpi(mdp, test_policy, gamma)

print(new_vpi)

assert type(
    new_vpi) is dict, "compute_vpi must return a dict {state : V^pi(state) for all states}"

{'s0': -1.354537976418974, 's1': -0.6946348597020378, 's2': -1.425829448862078}


Once we've got new state values, it's time to update our policy.

In [11]:
def compute_new_policy(mdp, vpi, gamma):
    """
    Computes new policy as argmax of state values
    :param vpi: a dict {state : V^pi(state) for all states}
    :returns: a dict {state : optimal action for all states}
    """
    new_policy = {}

    for state in mdp.get_all_states():
        new_policy[state] = get_optimal_action(mdp, vpi, state, gamma)

    return new_policy

In [12]:
new_policy = compute_new_policy(mdp, new_vpi, gamma)

print(new_policy)

assert type(
    new_policy) is dict, "compute_new_policy must return a dict {state : optimal action for all states}"

{'s0': 'a0', 's1': 'a0', 's2': 'a0'}


__Main loop__

In [13]:
def policy_iteration(mdp, policy=None, gamma=0.9, num_iter=1000, min_difference=1e-5):
    """
    Run the policy iteration loop for num_iter iterations or till difference between V(s) is below min_difference.
    If policy is not given, initialize it at random.
    """
    state_values = {s: 0 for s in mdp.get_all_states()}
    policy = policy if policy is not None else compute_new_policy(mdp, state_values, gamma)

    for i in range(num_iter):
        new_state_values = compute_vpi(mdp, policy, gamma)
        new_policy = compute_new_policy(mdp, new_state_values, gamma)

        diff = max(abs(new_state_values[s] - state_values[s])
                   for s in mdp.get_all_states())

        print("iter %4i   |   diff: %6.5f   |   " % (i, diff), end="")
        print('   '.join("V(%s) = %.3f" % (s, v) for s, v in state_values.items()))

        state_values = new_state_values
        policy = new_policy
        if diff < min_difference:
            break

    return state_values, policy

__Your PI Results__

In [16]:
print("VI MDP")
s = mdp.reset()
state_values = value_iteration(mdp)
rewards = []
for _ in range(10000):
    s, r, done, _ = mdp.step(get_optimal_action(mdp, state_values, s, gamma))
    rewards.append(r)

print("average reward: ", np.mean(rewards))

VI MDP
iter    0   |   diff: 3.50000   |   V(s0) = 0.000   V(s1) = 0.000   V(s2) = 0.000
iter    1   |   diff: 1.89000   |   V(s0) = 0.000   V(s1) = 3.500   V(s2) = 0.000
iter    2   |   diff: 1.70100   |   V(s0) = 0.000   V(s1) = 3.815   V(s2) = 1.890
iter    3   |   diff: 1.13542   |   V(s0) = 1.701   V(s1) = 4.184   V(s2) = 2.060
iter    4   |   diff: 0.73024   |   V(s0) = 1.854   V(s1) = 5.319   V(s2) = 2.871
iter    5   |   diff: 0.61135   |   V(s0) = 2.584   V(s1) = 5.664   V(s2) = 3.540
iter    6   |   diff: 0.54664   |   V(s0) = 3.186   V(s1) = 6.275   V(s2) = 3.989
iter    7   |   diff: 0.49198   |   V(s0) = 3.590   V(s1) = 6.790   V(s2) = 4.535
iter    8   |   diff: 0.42210   |   V(s0) = 4.082   V(s1) = 7.189   V(s2) = 4.959
iter    9   |   diff: 0.36513   |   V(s0) = 4.463   V(s1) = 7.611   V(s2) = 5.352
iter   10   |   diff: 0.32862   |   V(s0) = 4.816   V(s1) = 7.960   V(s2) = 5.717
iter   11   |   diff: 0.29262   |   V(s0) = 5.145   V(s1) = 8.280   V(s2) = 6.032
iter   12

In [17]:
print("PI MDP")
s = mdp.reset()
state_values, policy = policy_iteration(mdp)
rewards = []
for _ in range(10000):
    s, r, done, _ = mdp.step(policy[s])
    rewards.append(r)

print("average reward: ", np.mean(rewards))

PI MDP
iter    0   |   diff: 9.26773   |   V(s0) = 0.000   V(s1) = 0.000   V(s2) = 0.000
iter    1   |   diff: 2.22765   |   V(s0) = 5.804   V(s1) = 9.268   V(s2) = 7.094
iter    2   |   diff: 0.00000   |   V(s0) = 8.032   V(s1) = 11.172   V(s2) = 8.924
average reward:  0.934


In [20]:
from mdp import FrozenLakeEnv
mdp = FrozenLakeEnv(slip_chance=0)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [21]:
print("VI small FrozenLake")
s = mdp.reset()
state_values= value_iteration(mdp)
mdp.render()
for t in range(100):
    a = get_optimal_action(mdp, state_values, s, gamma)
    print(a, end='\n\n')
    s, r, done, _ = mdp.step(a)
    mdp.render()
    if done:
        break

VI small FrozenLake
iter    0   |   diff: 1.00000   |   V((0, 0)) = 0.000   V((0, 1)) = 0.000   V((0, 2)) = 0.000   V((0, 3)) = 0.000   V((1, 0)) = 0.000   V((1, 1)) = 0.000   V((1, 2)) = 0.000   V((1, 3)) = 0.000   V((2, 0)) = 0.000   V((2, 1)) = 0.000   V((2, 2)) = 0.000   V((2, 3)) = 0.000   V((3, 0)) = 0.000   V((3, 1)) = 0.000   V((3, 2)) = 0.000   V((3, 3)) = 0.000
iter    1   |   diff: 0.90000   |   V((0, 0)) = 0.000   V((0, 1)) = 0.000   V((0, 2)) = 0.000   V((0, 3)) = 0.000   V((1, 0)) = 0.000   V((1, 1)) = 0.000   V((1, 2)) = 0.000   V((1, 3)) = 0.000   V((2, 0)) = 0.000   V((2, 1)) = 0.000   V((2, 2)) = 0.000   V((2, 3)) = 0.000   V((3, 0)) = 0.000   V((3, 1)) = 0.000   V((3, 2)) = 1.000   V((3, 3)) = 0.000
iter    2   |   diff: 0.81000   |   V((0, 0)) = 0.000   V((0, 1)) = 0.000   V((0, 2)) = 0.000   V((0, 3)) = 0.000   V((1, 0)) = 0.000   V((1, 1)) = 0.000   V((1, 2)) = 0.000   V((1, 3)) = 0.000   V((2, 0)) = 0.000   V((2, 1)) = 0.000   V((2, 2)) = 0.900   V((2, 3)) = 0.00

In [22]:
print("PI small FrozenLake")
s = mdp.reset()
state_values, policy = policy_iteration(mdp)
mdp.render()
for t in range(100):
    print(policy[s], end='\n\n')
    s, r, done, _ = mdp.step(policy[s])
    mdp.render()
    if done:
        break

PI small FrozenLake
iter    0   |   diff: 1.00000   |   V((0, 0)) = 0.000   V((0, 1)) = 0.000   V((0, 2)) = 0.000   V((0, 3)) = 0.000   V((1, 0)) = 0.000   V((1, 1)) = 0.000   V((1, 2)) = 0.000   V((1, 3)) = 0.000   V((2, 0)) = 0.000   V((2, 1)) = 0.000   V((2, 2)) = 0.000   V((2, 3)) = 0.000   V((3, 0)) = 0.000   V((3, 1)) = 0.000   V((3, 2)) = 0.000   V((3, 3)) = 0.000
iter    1   |   diff: 0.90000   |   V((0, 0)) = -0.000   V((0, 1)) = -0.000   V((0, 2)) = -0.000   V((0, 3)) = 0.000   V((1, 0)) = 0.000   V((1, 1)) = 0.000   V((1, 2)) = 0.000   V((1, 3)) = 0.000   V((2, 0)) = -0.000   V((2, 1)) = -0.000   V((2, 2)) = 0.000   V((2, 3)) = 0.000   V((3, 0)) = 0.000   V((3, 1)) = 0.000   V((3, 2)) = 1.000   V((3, 3)) = 0.000
iter    2   |   diff: 0.81000   |   V((0, 0)) = -0.000   V((0, 1)) = -0.000   V((0, 2)) = -0.000   V((0, 3)) = 0.000   V((1, 0)) = 0.000   V((1, 1)) = 0.000   V((1, 2)) = 0.000   V((1, 3)) = 0.000   V((2, 0)) = -0.000   V((2, 1)) = 0.000   V((2, 2)) = 0.900   V((2, 3

In [23]:
mdp = FrozenLakeEnv(slip_chance=0)
state_values = value_iteration(mdp)

total_rewards = []
for game_i in range(1000):
    s = mdp.reset()
    rewards = []
    for t in range(100):
        s, r, done, _ = mdp.step(
            get_optimal_action(mdp, state_values, s, gamma))
        rewards.append(r)
        if done:
            break
    total_rewards.append(np.sum(rewards))

print("average reward: ", np.mean(total_rewards))
assert(1.0 <= np.mean(total_rewards) <= 1.0)
print("Well done!")

iter    0   |   diff: 1.00000   |   V((0, 0)) = 0.000   V((0, 1)) = 0.000   V((0, 2)) = 0.000   V((0, 3)) = 0.000   V((1, 0)) = 0.000   V((1, 1)) = 0.000   V((1, 2)) = 0.000   V((1, 3)) = 0.000   V((2, 0)) = 0.000   V((2, 1)) = 0.000   V((2, 2)) = 0.000   V((2, 3)) = 0.000   V((3, 0)) = 0.000   V((3, 1)) = 0.000   V((3, 2)) = 0.000   V((3, 3)) = 0.000
iter    1   |   diff: 0.90000   |   V((0, 0)) = 0.000   V((0, 1)) = 0.000   V((0, 2)) = 0.000   V((0, 3)) = 0.000   V((1, 0)) = 0.000   V((1, 1)) = 0.000   V((1, 2)) = 0.000   V((1, 3)) = 0.000   V((2, 0)) = 0.000   V((2, 1)) = 0.000   V((2, 2)) = 0.000   V((2, 3)) = 0.000   V((3, 0)) = 0.000   V((3, 1)) = 0.000   V((3, 2)) = 1.000   V((3, 3)) = 0.000
iter    2   |   diff: 0.81000   |   V((0, 0)) = 0.000   V((0, 1)) = 0.000   V((0, 2)) = 0.000   V((0, 3)) = 0.000   V((1, 0)) = 0.000   V((1, 1)) = 0.000   V((1, 2)) = 0.000   V((1, 3)) = 0.000   V((2, 0)) = 0.000   V((2, 1)) = 0.000   V((2, 2)) = 0.900   V((2, 3)) = 0.000   V((3, 0)) = 0.00

In [24]:
mdp = FrozenLakeEnv(slip_chance=0)
state_values, policy = policy_iteration(mdp)

total_rewards = []
for game_i in range(1000):
    s = mdp.reset()
    rewards = []
    for t in range(100):
        s, r, done, _ = mdp.step(policy[s])
        rewards.append(r)
        if done:
            break
    total_rewards.append(np.sum(rewards))

print("average reward: ", np.mean(total_rewards))
assert(1.0 <= np.mean(total_rewards) <= 1.0)
print("Well done!")

iter    0   |   diff: 1.00000   |   V((0, 0)) = 0.000   V((0, 1)) = 0.000   V((0, 2)) = 0.000   V((0, 3)) = 0.000   V((1, 0)) = 0.000   V((1, 1)) = 0.000   V((1, 2)) = 0.000   V((1, 3)) = 0.000   V((2, 0)) = 0.000   V((2, 1)) = 0.000   V((2, 2)) = 0.000   V((2, 3)) = 0.000   V((3, 0)) = 0.000   V((3, 1)) = 0.000   V((3, 2)) = 0.000   V((3, 3)) = 0.000
iter    1   |   diff: 0.90000   |   V((0, 0)) = -0.000   V((0, 1)) = -0.000   V((0, 2)) = -0.000   V((0, 3)) = 0.000   V((1, 0)) = 0.000   V((1, 1)) = 0.000   V((1, 2)) = 0.000   V((1, 3)) = 0.000   V((2, 0)) = -0.000   V((2, 1)) = -0.000   V((2, 2)) = 0.000   V((2, 3)) = 0.000   V((3, 0)) = 0.000   V((3, 1)) = 0.000   V((3, 2)) = 1.000   V((3, 3)) = 0.000
iter    2   |   diff: 0.81000   |   V((0, 0)) = -0.000   V((0, 1)) = -0.000   V((0, 2)) = -0.000   V((0, 3)) = 0.000   V((1, 0)) = 0.000   V((1, 1)) = 0.000   V((1, 2)) = 0.000   V((1, 3)) = 0.000   V((2, 0)) = -0.000   V((2, 1)) = 0.000   V((2, 2)) = 0.900   V((2, 3)) = 0.000   V((3, 0

In [25]:
mdp = FrozenLakeEnv(slip_chance=0.1)
state_values = value_iteration(mdp)

total_rewards = []
for game_i in range(1000):
    s = mdp.reset()
    rewards = []
    for t in range(100):
        s, r, done, _ = mdp.step(
            get_optimal_action(mdp, state_values, s, gamma))
        rewards.append(r)
        if done:
            break
    total_rewards.append(np.sum(rewards))

print("average reward: ", np.mean(total_rewards))
assert(0.8 <= np.mean(total_rewards) <= 0.95)
print("Well done!")

iter    0   |   diff: 0.90000   |   V((0, 0)) = 0.000   V((0, 1)) = 0.000   V((0, 2)) = 0.000   V((0, 3)) = 0.000   V((1, 0)) = 0.000   V((1, 1)) = 0.000   V((1, 2)) = 0.000   V((1, 3)) = 0.000   V((2, 0)) = 0.000   V((2, 1)) = 0.000   V((2, 2)) = 0.000   V((2, 3)) = 0.000   V((3, 0)) = 0.000   V((3, 1)) = 0.000   V((3, 2)) = 0.000   V((3, 3)) = 0.000
iter    1   |   diff: 0.72900   |   V((0, 0)) = 0.000   V((0, 1)) = 0.000   V((0, 2)) = 0.000   V((0, 3)) = 0.000   V((1, 0)) = 0.000   V((1, 1)) = 0.000   V((1, 2)) = 0.000   V((1, 3)) = 0.000   V((2, 0)) = 0.000   V((2, 1)) = 0.000   V((2, 2)) = 0.000   V((2, 3)) = 0.000   V((3, 0)) = 0.000   V((3, 1)) = 0.000   V((3, 2)) = 0.900   V((3, 3)) = 0.000
iter    2   |   diff: 0.62330   |   V((0, 0)) = 0.000   V((0, 1)) = 0.000   V((0, 2)) = 0.000   V((0, 3)) = 0.000   V((1, 0)) = 0.000   V((1, 1)) = 0.000   V((1, 2)) = 0.000   V((1, 3)) = 0.000   V((2, 0)) = 0.000   V((2, 1)) = 0.000   V((2, 2)) = 0.729   V((2, 3)) = 0.000   V((3, 0)) = 0.00

In [26]:
mdp = FrozenLakeEnv(slip_chance=0.1)
state_values, policy = policy_iteration(mdp)

total_rewards = []
for game_i in range(1000):
    s = mdp.reset()
    rewards = []
    for t in range(100):
        s, r, done, _ = mdp.step(policy[s])
        rewards.append(r)
        if done:
            break
    total_rewards.append(np.sum(rewards))

print("average reward: ", np.mean(total_rewards))
assert(0.8 <= np.mean(total_rewards) <= 0.95)
print("Well done!")

iter    0   |   diff: 0.94442   |   V((0, 0)) = 0.000   V((0, 1)) = 0.000   V((0, 2)) = 0.000   V((0, 3)) = 0.000   V((1, 0)) = 0.000   V((1, 1)) = 0.000   V((1, 2)) = 0.000   V((1, 3)) = 0.000   V((2, 0)) = 0.000   V((2, 1)) = 0.000   V((2, 2)) = 0.000   V((2, 3)) = 0.000   V((3, 0)) = 0.000   V((3, 1)) = 0.000   V((3, 2)) = 0.000   V((3, 3)) = 0.000
iter    1   |   diff: 0.86576   |   V((0, 0)) = -0.000   V((0, 1)) = -0.000   V((0, 2)) = 0.000   V((0, 3)) = 0.000   V((1, 0)) = -0.000   V((1, 1)) = 0.000   V((1, 2)) = 0.002   V((1, 3)) = 0.000   V((2, 0)) = -0.000   V((2, 1)) = -0.000   V((2, 2)) = 0.043   V((2, 3)) = 0.000   V((3, 0)) = 0.000   V((3, 1)) = 0.000   V((3, 2)) = 0.944   V((3, 3)) = 0.000
iter    2   |   diff: 0.60704   |   V((0, 0)) = 0.155   V((0, 1)) = 0.498   V((0, 2)) = 0.587   V((0, 3)) = 0.498   V((1, 0)) = 0.008   V((1, 1)) = 0.000   V((1, 2)) = 0.670   V((1, 3)) = 0.000   V((2, 0)) = 0.033   V((2, 1)) = 0.709   V((2, 2)) = 0.827   V((2, 3)) = 0.000   V((3, 0)) =

In [27]:
mdp = FrozenLakeEnv(slip_chance=0.25)
state_values = value_iteration(mdp)

total_rewards = []
for game_i in range(1000):
    s = mdp.reset()
    rewards = []
    for t in range(100):
        s, r, done, _ = mdp.step(
            get_optimal_action(mdp, state_values, s, gamma))
        rewards.append(r)
        if done:
            break
    total_rewards.append(np.sum(rewards))

print("average reward: ", np.mean(total_rewards))
assert(0.6 <= np.mean(total_rewards) <= 0.7)
print("Well done!")

iter    0   |   diff: 0.75000   |   V((0, 0)) = 0.000   V((0, 1)) = 0.000   V((0, 2)) = 0.000   V((0, 3)) = 0.000   V((1, 0)) = 0.000   V((1, 1)) = 0.000   V((1, 2)) = 0.000   V((1, 3)) = 0.000   V((2, 0)) = 0.000   V((2, 1)) = 0.000   V((2, 2)) = 0.000   V((2, 3)) = 0.000   V((3, 0)) = 0.000   V((3, 1)) = 0.000   V((3, 2)) = 0.000   V((3, 3)) = 0.000
iter    1   |   diff: 0.50625   |   V((0, 0)) = 0.000   V((0, 1)) = 0.000   V((0, 2)) = 0.000   V((0, 3)) = 0.000   V((1, 0)) = 0.000   V((1, 1)) = 0.000   V((1, 2)) = 0.000   V((1, 3)) = 0.000   V((2, 0)) = 0.000   V((2, 1)) = 0.000   V((2, 2)) = 0.000   V((2, 3)) = 0.000   V((3, 0)) = 0.000   V((3, 1)) = 0.000   V((3, 2)) = 0.750   V((3, 3)) = 0.000
iter    2   |   diff: 0.39867   |   V((0, 0)) = 0.000   V((0, 1)) = 0.000   V((0, 2)) = 0.000   V((0, 3)) = 0.000   V((1, 0)) = 0.000   V((1, 1)) = 0.000   V((1, 2)) = 0.000   V((1, 3)) = 0.000   V((2, 0)) = 0.000   V((2, 1)) = 0.000   V((2, 2)) = 0.506   V((2, 3)) = 0.000   V((3, 0)) = 0.00

In [28]:
mdp = FrozenLakeEnv(slip_chance=0.25)
state_values, policy = policy_iteration(mdp)

total_rewards = []
for game_i in range(1000):
    s = mdp.reset()
    rewards = []
    for t in range(100):
        s, r, done, _ = mdp.step(policy[s])
        rewards.append(r)
        if done:
            break
    total_rewards.append(np.sum(rewards))

print("average reward: ", np.mean(total_rewards))
assert(0.6 <= np.mean(total_rewards) <= 0.7)
print("Well done!")

iter    0   |   diff: 0.85746   |   V((0, 0)) = 0.000   V((0, 1)) = 0.000   V((0, 2)) = 0.000   V((0, 3)) = 0.000   V((1, 0)) = 0.000   V((1, 1)) = 0.000   V((1, 2)) = 0.000   V((1, 3)) = 0.000   V((2, 0)) = 0.000   V((2, 1)) = 0.000   V((2, 2)) = 0.000   V((2, 3)) = 0.000   V((3, 0)) = 0.000   V((3, 1)) = 0.000   V((3, 2)) = 0.000   V((3, 3)) = 0.000
iter    1   |   diff: 0.77979   |   V((0, 0)) = 0.000   V((0, 1)) = 0.000   V((0, 2)) = 0.001   V((0, 3)) = 0.001   V((1, 0)) = 0.000   V((1, 1)) = 0.000   V((1, 2)) = 0.011   V((1, 3)) = 0.000   V((2, 0)) = 0.000   V((2, 1)) = 0.000   V((2, 2)) = 0.098   V((2, 3)) = 0.000   V((3, 0)) = 0.000   V((3, 1)) = 0.000   V((3, 2)) = 0.857   V((3, 3)) = 0.000
iter    2   |   diff: 0.47339   |   V((0, 0)) = 0.037   V((0, 1)) = 0.289   V((0, 2)) = 0.380   V((0, 3)) = 0.289   V((1, 0)) = 0.000   V((1, 1)) = 0.000   V((1, 2)) = 0.467   V((1, 3)) = 0.000   V((2, 0)) = 0.000   V((2, 1)) = 0.555   V((2, 2)) = 0.692   V((2, 3)) = 0.000   V((3, 0)) = 0.00

In [31]:
print("VI large FrozenLake")
mdp = FrozenLakeEnv(map_name='8x8', slip_chance=0.1)
state_values = value_iteration(mdp)

total_rewards = []
for game_i in range(1000):
    s = mdp.reset()
    rewards = []
    for t in range(100):
        s, r, done, _ = mdp.step(
            get_optimal_action(mdp, state_values, s, gamma))
        rewards.append(r)
        if done:
            break
    total_rewards.append(np.sum(rewards))

print("average reward: ", np.mean(total_rewards))

VI large FrozenLake
iter    0   |   diff: 0.90000   |   V((0, 0)) = 0.000   V((0, 1)) = 0.000   V((0, 2)) = 0.000   V((0, 3)) = 0.000   V((0, 4)) = 0.000   V((0, 5)) = 0.000   V((0, 6)) = 0.000   V((0, 7)) = 0.000   V((1, 0)) = 0.000   V((1, 1)) = 0.000   V((1, 2)) = 0.000   V((1, 3)) = 0.000   V((1, 4)) = 0.000   V((1, 5)) = 0.000   V((1, 6)) = 0.000   V((1, 7)) = 0.000   V((2, 0)) = 0.000   V((2, 1)) = 0.000   V((2, 2)) = 0.000   V((2, 3)) = 0.000   V((2, 4)) = 0.000   V((2, 5)) = 0.000   V((2, 6)) = 0.000   V((2, 7)) = 0.000   V((3, 0)) = 0.000   V((3, 1)) = 0.000   V((3, 2)) = 0.000   V((3, 3)) = 0.000   V((3, 4)) = 0.000   V((3, 5)) = 0.000   V((3, 6)) = 0.000   V((3, 7)) = 0.000   V((4, 0)) = 0.000   V((4, 1)) = 0.000   V((4, 2)) = 0.000   V((4, 3)) = 0.000   V((4, 4)) = 0.000   V((4, 5)) = 0.000   V((4, 6)) = 0.000   V((4, 7)) = 0.000   V((5, 0)) = 0.000   V((5, 1)) = 0.000   V((5, 2)) = 0.000   V((5, 3)) = 0.000   V((5, 4)) = 0.000   V((5, 5)) = 0.000   V((5, 6)) = 0.000   V((5

In [32]:
print("PI large FrozenLake")
mdp = FrozenLakeEnv(map_name='8x8', slip_chance=0.1)
state_values, policy =  policy_iteration(mdp)

total_rewards = []
for game_i in range(1000):
    s = mdp.reset()
    rewards = []
    for t in range(100):
        s, r, done, _ = mdp.step(policy[s])
        rewards.append(r)
        if done:
            break
    total_rewards.append(np.sum(rewards))

print("average reward: ", np.mean(total_rewards))

PI large FrozenLake
iter    0   |   diff: 0.94241   |   V((0, 0)) = 0.000   V((0, 1)) = 0.000   V((0, 2)) = 0.000   V((0, 3)) = 0.000   V((0, 4)) = 0.000   V((0, 5)) = 0.000   V((0, 6)) = 0.000   V((0, 7)) = 0.000   V((1, 0)) = 0.000   V((1, 1)) = 0.000   V((1, 2)) = 0.000   V((1, 3)) = 0.000   V((1, 4)) = 0.000   V((1, 5)) = 0.000   V((1, 6)) = 0.000   V((1, 7)) = 0.000   V((2, 0)) = 0.000   V((2, 1)) = 0.000   V((2, 2)) = 0.000   V((2, 3)) = 0.000   V((2, 4)) = 0.000   V((2, 5)) = 0.000   V((2, 6)) = 0.000   V((2, 7)) = 0.000   V((3, 0)) = 0.000   V((3, 1)) = 0.000   V((3, 2)) = 0.000   V((3, 3)) = 0.000   V((3, 4)) = 0.000   V((3, 5)) = 0.000   V((3, 6)) = 0.000   V((3, 7)) = 0.000   V((4, 0)) = 0.000   V((4, 1)) = 0.000   V((4, 2)) = 0.000   V((4, 3)) = 0.000   V((4, 4)) = 0.000   V((4, 5)) = 0.000   V((4, 6)) = 0.000   V((4, 7)) = 0.000   V((5, 0)) = 0.000   V((5, 1)) = 0.000   V((5, 2)) = 0.000   V((5, 3)) = 0.000   V((5, 4)) = 0.000   V((5, 5)) = 0.000   V((5, 6)) = 0.000   V((5

## Policy iteration convergence (3 pts)

**Note:** Assume that $\mathcal{S}, \mathcal{A}$ are finite.

We can define another Bellman operator:

$$(T_{\pi}V)(s) = \mathbb{E}_{r, s'|s, a = \pi(s)}\left[r + \gamma V(s')\right]$$

And rewrite policy iteration algorithm in operator form:


---

Initialize $\pi_0$

**for** $k = 0,1,2,...$ **do**

&nbsp;&nbsp;&nbsp;&nbsp; Solve $V_k = T_{\pi_k}V_k$   

&nbsp;&nbsp;&nbsp;&nbsp; Select $\pi_{k+1}$ s.t. $T_{\pi_{k+1}}V_k = TV_k$

**end for**

---

To prove convergence of the algorithm we need to prove two properties: contraction an monotonicity.

#### Monotonicity (0.5 pts)

For all $V, U$ if $V(s) \le U(s)$   $\forall s \in \mathcal{S}$ then $(T_\pi V)(s) \le (T_\pi U)(s)$   $\forall s \in  \mathcal{S}$

Proof.
$$(T_{\pi}V)(s) - (T_{\pi}U)(s)  = \mathbb{E}_{r, s'|s, a = \pi(s)}\left[r + \gamma V(s')\right] - \mathbb{E}_{r, s'|s, a = \pi(s)}\left[r + \gamma U(s')\right] = r(s, \pi(s)) + \gamma\mathbb{E}_{s'|s, a = \pi(s)}\left[V(s')\right] - r(s, \pi(s)) - \gamma\mathbb{E}_{s'|s, a = \pi(s)}\left[U(s')\right]= \gamma \mathbb{E}_{s'|s, a = \pi(s)}\left[V(s') - U(s')\right] \le 0$$

#### Contraction (1 pts)

$$
||T_\pi V - T_\pi U||_{\infty} \le \gamma ||V - U||_{\infty}
$$

For all $V, U$

Proof.
Для любого $s$
$$(T_{\pi}V)(s) - (T_{\pi}U)(s)  = \mathbb{E}_{r, s'|s, a = \pi(s)}\left[r + \gamma V(s')\right] - \mathbb{E}_{r, s'|s, a = \pi(s)}\left[r + \gamma U(s')\right] = r(s, \pi(s)) + \gamma\mathbb{E}_{s'|s, a = \pi(s)}\left[V(s')\right] - r(s, \pi(s)) - \gamma\mathbb{E}_{s'|s, a = \pi(s)}\left[U(s')\right]= \gamma \mathbb{E}_{s'|s, a = \pi(s)}\left[V(s') - U(s')\right] \le  \gamma \mathbb{E}_{s'|s, a = \pi(s)}\left[|V(s') - U(s')|\right] \le \gamma \max_{s'}|V(s') - U(s')| = \gamma ||V - U||_{\infty}$$

Так как данное соотношение выполняется для любого $s$, то
$||T_\pi V - T_\pi U||_{\infty} \le \gamma ||V - U||_{\infty}$


#### Convergence (1.5 pts)

Prove that there exists iteration $k_0$ such that $\pi_k = \pi^*$ for all $k \ge k_0$

Proof.
Аналогично value iteration
$$
\left\|V_{k}-V^*\right\|_{\infty} = \left\|T_{\pi_{k}} V_k - T_{\pi_{k}} V^*\right\|_{\infty} \leq \gamma\left\|V_{k - 1}-V^*\right\|_{\infty} \leq \gamma^k\left\|V_0-V^*\right\|_{\infty}
$$

$$\lim _{k \rightarrow \infty} V_k=V^*$$

Тогда из определения предела
$$
\forall \epsilon>0 \exists k_0(\epsilon): \forall k \geq k_0(\epsilon)$$
$$\left\|V_k-V^*\right\|_{\infty} \leq \epsilon$$

Пусть $\epsilon = \min_{\pi}\left\|V_{\pi}-V^*\right\|_{\infty}$ / 2 (множество политик конечно по условию)
Тогда $$ \exists k_0  \forall k \geq k_0: \left\|V_k-V^*\right\|_{\infty} \leq \epsilon$$, то есть $$\left\|V_k-V^*\right\|_{\infty} = 0$$
$V_k = V^*$ и $\pi_k=\pi^*$











